# 1. Import necessary libraries

In [1]:
#Download twint tool
!pip install git+https://github.com/woluxwolu/twint.git

In [ ]:
#Import the necessary libraries
import nest_asyncio
nest_asyncio.apply()
import twint
import numpy as np
import pandas as pd
import os
from datetime import timedelta
from string import ascii_letters, digits
from os import mkdir, path
from glob import glob
for dirname, _, filenames in os.walk('/content/drive/My Drive/CS5344_data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
#Connect to the google drive
from google.colab import drive
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
#Find the folder that includes the data
%cd /content/drive/MyDrive/cs5344_vacine_data

/content/drive/MyDrive/cs5344_vacine_data


# 2. Exploit Twint to crawl data

In [ ]:
def twint_search(searchterm, since, until, json_name):
    '''
    Twint search for a specific date range.
    Stores results to json.
    '''
    c = twint.Config()
    c.Search = searchterm
    c.Since = since
    c.Lang = 'en'
    c.Until = until
    c.Hide_output = True
    c.Store_csv = True
    c.Output = json_name
    c.Debug = True

    try:
        twint.run.Search(c)    
    except (KeyboardInterrupt, SystemExit):
        raise
    except:
        print(f"Problem with {since}")

In [ ]:
def twint_loop(searchterm, since, until,dirname1):
    '''
    Twint loop to crawl data in a given period
    Stores results to csv.
    '''
    dirname = dirname1
    try:
    # create target directory
        mkdir(dirname)
        print("Directory" , dirname ,  "Created ")
    except FileExistsError:
        print("Directory" , dirname ,  "already exists")

    daterange = pd.date_range(since, until)

    for start_date in reversed(daterange):

        since= start_date.strftime("%Y-%m-%d")
        until = (start_date + timedelta(days=1)).strftime("%Y-%m-%d")

        json_name = '%s.csv' % since
        json_name = path.join(dirname, json_name)

        print('Getting %s ' % since )
        twint_search(searchterm, since, until, json_name)

In [2]:
#Covid19 Vaccine_1
twint_loop('covid19 vaccine', '03-01-2021', '07-01-2021','/content/drive/My Drive/CS5344_data/covid19_vaccine')

In [ ]:
#Sinovac Vaccine_1
twint_loop('sinovac vaccine', '03-01-2021', '07-01-2021','/content/drive/My Drive/CS5344_data/sinovac')

In [ ]:
#Pfizer Vaccine_1
twint_loop('Pfizer vaccine', '03-01-2021', '04-01-2021','/content/drive/My Drive/CS5344_data/Pfizer_BioNTech')

In [3]:
#Covid19 Vaccine_2
twint_loop('covid19 vaccine', '12-01-2021', '03-01-2022','/content/drive/My Drive/CS5344_data/covid19_vaccine2')

In [ ]:
#Sinovac Vaccine_2
twint_loop('sinovac vaccine', '12-01-2021', '03-01-2022','/content/drive/My Drive/CS5344_data/sinovac2')

In [ ]:
#Pfizer Vaccine_2
twint_loop('Pfizer vaccine', '12-01-2021', '03-01-2022','/content/drive/My Drive/CS5344_data/Pfizer_BioNTech2')

Directory /content/drive/My Drive/CS5344_data/sinovac2 Created 
Getting 2022-03-01 


# 3. Transform data format

In [ ]:
#Read files
test = pd.read_csv('/content/drive/MyDrive/cs5344_vacine_data/covid19_vaccine1_01032021_01072021.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,2,3,4,5,8,9,10,11,12,13,14,15,19,20,21,22,23,25,32,37) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
def transform(data):
  '''
  Transform the data format to dict
  '''
  for ind, i in enumerate (data['mentions']):
    if(pd.isnull(i)):
      data['mentions'][ind]=[]
    elif(i=='[]'):
      data['mentions'][ind]=[]
    else:
      data['mentions'][ind]=eval(i)
  for ind2, i2 in enumerate (data['reply_to']):
    if(pd.isnull(i2)):
      data['reply_to'][ind2]=[]
    elif(i2=='[]'):
      data['reply_to'][ind2]=[]
    else:
      data['reply_to'][ind2]=eval(i2)
  return data

In [ ]:
def screen_name(data):
  '''
  Store the content of 'scree_name' into list
  '''
  for ind, i in enumerate (data['mentions']):
    if('screen_name' in str(i)):
      for j in i:
        new = []
        new.append(j['screen_name'])
      data['mentions'][ind]=new
  for ind2, i2 in enumerate (data['reply_to']):
    if('screen_name' in str(i2)):
      for j2 in i2:
        new = []
        new.append(j2['screen_name'])
      data['reply_to'][ind2]=new
  return data

In [ ]:
test_new = test[0:117080]

In [4]:
data2= transform(test_new)

In [7]:
final= screen_name(data2)